In [ ]:
import io
import numpy as np
import pandas as pd

In [ ]:
train_path = '/content/drive/MyDrive/Project 3/Social media text/oct27.train'
dev_path = '/content/drive/MyDrive/Project 3/Social media text/oct27.dev'
test_path = '/content/drive/MyDrive/Project 3/Social media text/oct27.test'

In [ ]:
with io.open(train_path, 'r') as f:
  train_data = f.readlines()
with io.open(dev_path, 'r') as f1:
  dev_data = f1.readlines()
with io.open(test_path, 'r') as f2:
  test_data = f2.readlines()
print(train_data[0])
print(dev_data[0])
print(test_data[0])

I	O

@ciaranyree	@

RT	~



In [ ]:
train_data[:50]

['I\tO\n',
 'predict\tV\n',
 'I\tO\n',
 "won't\tV\n",
 'win\tV\n',
 'a\tD\n',
 'single\tA\n',
 'game\tN\n',
 'I\tO\n',
 'bet\tV\n',
 'on\tP\n',
 '.\t,\n',
 'Got\tV\n',
 'Cliff\t^\n',
 'Lee\t^\n',
 'today\tN\n',
 ',\t,\n',
 'so\tP\n',
 'if\tP\n',
 'he\tO\n',
 'loses\tV\n',
 'its\tL\n',
 'on\tP\n',
 'me\tO\n',
 'RT\t~\n',
 '@e_one\t@\n',
 ':\t~\n',
 'Texas\t^\n',
 '(\t,\n',
 'cont\t~\n',
 ')\t,\n',
 'http://tl.gd/6meogh\tU\n',
 '\n',
 'RT\t~\n',
 '@DjBlack_Pearl\t@\n',
 ':\t~\n',
 'wat\tO\n',
 'muhfuckaz\tN\n',
 'wearin\tV\n',
 '4\tP\n',
 'the\tD\n',
 'lingerie\tN\n',
 'party\tN\n',
 '?????\t,\n',
 '\n',
 'Wednesday\t^\n',
 '27th\tA\n',
 'october\t^\n',
 '2010\t$\n',
 '.\t,\n']

In [ ]:
def get_data(data):
  sentences = []
  labels = []
  tokens = []
  tags = []
  for toktag in data:
    if len(toktag.split()) == 2:
      tokens.append(toktag.split()[0])
      tags.append(toktag.split()[1])
    if len(toktag.split()) == 1:
      tokens.append(" ")
      tags.append(toktag.split()[0])
    if not toktag.split():
      sentences.append(tokens)
      labels.append(tags)
      tokens = []
      tags = []
  return sentences, labels

In [ ]:
train_sent, train_labels = get_data(train_data)
dev_sent, dev_labels = get_data(dev_data)
test_sent, test_labels = get_data(test_data)

In [ ]:
print(train_sent[0])
print(train_labels[0])

['I', 'predict', 'I', "won't", 'win', 'a', 'single', 'game', 'I', 'bet', 'on', '.', 'Got', 'Cliff', 'Lee', 'today', ',', 'so', 'if', 'he', 'loses', 'its', 'on', 'me', 'RT', '@e_one', ':', 'Texas', '(', 'cont', ')', 'http://tl.gd/6meogh']
['O', 'V', 'O', 'V', 'V', 'D', 'A', 'N', 'O', 'V', 'P', ',', 'V', '^', '^', 'N', ',', 'P', 'P', 'O', 'V', 'L', 'P', 'O', '~', '@', '~', '^', ',', '~', ',', 'U']


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.9MB/s 
     |████████████████████████████████| 890kB 17.1MB/s 
     |████████████████████████████████| 2.9MB 43.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c9be15a898c2bde2d15fa6639beb0ab200feae0196014cc116adf6002e292a2d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets

from transformers import BertTokenizer, BertModel

import numpy as np
import time
import random
import functools

In [ ]:
# set the random seeds for reproducability
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [ ]:
init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, pad_token, unk_token)

[CLS] [PAD] [UNK]


In [ ]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, pad_token_idx, unk_token_idx)

101 0 100


In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-large-uncased']

print(max_input_length)

512


In [ ]:
def cut_and_convert_to_id(tokens, tokenizer, max_input_length):
    tokens = tokens[:max_input_length-1]
    tokens = tokenizer.convert_tokens_to_ids(tokens)
    return tokens

In [ ]:
def cut_to_max_length(tokens, max_input_length):
    tokens = tokens[:max_input_length-1]
    return tokens

In [ ]:
text_preprocessor = functools.partial(cut_and_convert_to_id,
                                      tokenizer = tokenizer,
                                      max_input_length = max_input_length)

tag_preprocessor = functools.partial(cut_to_max_length,
                                     max_input_length = max_input_length)

In [ ]:
TEXT = data.Field(use_vocab = False,
                  lower = True,
                  preprocessing = text_preprocessor,
                  init_token = init_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

TWEET_TAGS = data.Field(unk_token = None,
                     init_token = '<pad>',
                     preprocessing = tag_preprocessor)

In [ ]:
fields = [(None, None), ("text", TEXT), ("label", TWEET_TAGS)]

In [ ]:
text_col = [' '.join([x for x in sent]) for sent in train_sent]
label_col = [' '.join([x for x in lab]) for lab in train_labels]
dict = {'text': text_col, 'label': label_col}
df = pd.DataFrame(dict)
df.head()

,text,label
0,I predict I won't win a single game I bet on ....,"O V O V V D A N O V P , V ^ ^ N , P P O V L P ..."
1,RT @DjBlack_Pearl : wat muhfuckaz wearin 4 the...,"~ @ ~ O N V P D N N ,"
2,Wednesday 27th october 2010 . 》have a nice day :),"^ A ^ $ , V D A N E"
3,"RT @ddlovato : @joejonas oh , hey THANKS jerk !","~ @ ~ @ ! , ! N N ,"
4,"@thecamion I like monkeys , but I still hate C...","@ O V N , & O R V ^ N N ,"


In [ ]:
df.to_csv('/content/drive/MyDrive/Project 3/Social media text/train.csv')

In [ ]:
text_col = [' '.join([x for x in sent]) for sent in dev_sent]
label_col = [' '.join([x for x in lab]) for lab in dev_labels]
dict = {'text': text_col, 'label': label_col}
df = pd.DataFrame(dict)
df.head(10)

,text,label
0,"@ciaranyree it was on football wives , one of ...","@ O V P N N , $ P D N & D N V ^ ^"
1,RT @TheRealQuailman : Currently laughing at La...,"~ @ ~ R V P ^ N ,"
2,RT @HollywoodOompa : Sat November 6 ill be at ...,~ @ ~ ^ ^ $ L V P ^ N N P D N N N P ~ N L D N ...
3,You don't know my struggle,O V V D N
4,"Wind 2.0 mph SE . Barometer 29.676 in , Rising...","N $ N N , N $ N , V , N $ ^ , N N $ N , N $"
5,@ShiversTheNinja forgive me for blowing up you...,"@ V O P V T D ^ N N ,"
6,New Question : How CAN you mend a broken heart...,"A N , R V O V D A N , R , V , U"
7,@justinbieber can u follow me please ??? it wi...,"@ V O V O V , O V V D N P O , E E"
8,@JoycieeLovesUuu =) Lls,@ E !
9,"@carolyncallahan I knew about it last night , ...","@ O V P O A N , & V V V ^ P L R V V P O D N P ..."


In [ ]:
df.to_csv('/content/drive/MyDrive/Project 3/Social media text/dev.csv')

In [ ]:
text_col = [' '.join([x for x in sent]) for sent in test_sent]
label_col = [' '.join([x for x in lab]) for lab in test_labels]
dict = {'text': text_col, 'label': label_col}
df = pd.DataFrame(dict)
df.head(10)

,text,label
0,RT @yomonroe : i want sex_,~ @ ~ O V N
1,My Tummy Feels Funny .. EMILY ! What Are You D...,"D N V A , ^ , O V O V P R , E"
2,Down in the basement with @1cierra22 @lola_sym...,P P D N P @ @ @
3,RT @mtnridge : We are selling Christmas trees ...,"~ @ ~ O V V ^ N , V D N U & V T N P D G ~"
4,I think when I am done with my MFA I should te...,O V R O V V P D N O V V O P N & V D A N P D N ...
5,@john_mason_ Is there a Flex for CF or Dreamwe...,"@ V X D ^ P ^ & ^ P ^ N R , #"
6,I just became the mayor of Off 5th Saks Fifth ...,"O R V D N P ^ ^ ^ ^ ^ N P @ , U"
7,RT @HuffPostCollege : The Lowdown On Four Loko...,"~ @ ~ D N P ^ ^ , $ N R P V N N , P P O V O , U"
8,Some ppl r juz unrealiable .,"D N V R A ,"
9,@kristenmchugh22 @nerdsinbabeland I'm actually...,"@ @ L R D A ^ ^ N , L D A ^ N"


In [ ]:
df.to_csv('/content/drive/MyDrive/Project 3/Social media text/test.csv')

In [ ]:
from torchtext.data import TabularDataset
train_data, dev_data, test_data = TabularDataset.splits(
    path='/content/drive/MyDrive/Project 3/Social media text',
    train='train.csv', validation='dev.csv', test='test.csv',
    format='csv',
    skip_header=True,
    fields=fields
)

In [ ]:
print(vars(train_data.examples[0]))

{'text': [1045, 16014, 1045, 100, 2663, 1037, 2309, 2208, 1045, 6655, 2006, 1012, 2288, 7656, 3389, 2651, 1010, 2061, 2065, 2002, 12386, 2049, 2006, 2033, 19387, 100, 1024, 3146, 1006, 100, 1007, 100], 'label': ['O', 'V', 'O', 'V', 'V', 'D', 'A', 'N', 'O', 'V', 'P', ',', 'V', '^', '^', 'N', ',', 'P', 'P', 'O', 'V', 'L', 'P', 'O', '~', '@', '~', '^', ',', '~', ',', 'U']}


In [ ]:
TWEET_TAGS.build_vocab(train_data)
print(TWEET_TAGS.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fa3cb3f3620>, {'<pad>': 0, 'V': 1, 'N': 2, ',': 3, 'P': 4, 'O': 5, '^': 6, 'D': 7, 'A': 8, '@': 9, 'R': 10, '~': 11, '!': 12, 'L': 13, '&': 14, 'U': 15, '$': 16, 'E': 17, '#': 18, 'G': 19, 'T': 20, 'Z': 21, 'S': 22, 'X': 23, 'M': 24, 'Y': 25})


In [ ]:
BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, dev_data, test_data),
    sort=False,
    batch_size=BATCH_SIZE,
    device = device
)

In [ ]:
print(device)

cuda


In [ ]:
class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
  
        #text = [sent len, batch size]
    
        text = text.permute(1, 0)
        
        #text = [batch size, sent len]
        
        embedded = self.dropout(self.bert(text)[0])
        
        #embedded = [batch size, seq len, emb dim]
                
        embedded = embedded.permute(1, 0, 2)
                    
        #embedded = [sent len, batch size, emb dim]
        
        predictions = self.fc(self.dropout(embedded))
        
        #predictions = [sent len, batch size, output dim]
        
        return predictions

In [ ]:
bert = BertModel.from_pretrained('bert-large-uncased')

In [ ]:
OUTPUT_DIM = len(TWEET_TAGS.vocab)
DROPOUT = 0.25

model = BERTPoSTagger(bert,
                      OUTPUT_DIM, 
                      DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 335,168,538 trainable parameters


In [ ]:
LEARNING_RATE = 3e-5

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
TAG_PAD_IDX = TWEET_TAGS.vocab.stoi[TWEET_TAGS.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TAG_PAD_IDX)

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y, tag_pad_idx):
    
    max_preds = preds.argmax(dim = 1, keepdim = True) 
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

In [ ]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.label
                
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.label
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/Project 3/Social media text/tweet_BERT.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  


Epoch: 01 | Epoch Time: 0m 25s
	Train Loss: 2.673 | Train Acc: 19.78%
	 Val. Loss: 2.236 |  Val. Acc: 41.61%
Epoch: 02 | Epoch Time: 0m 24s
	Train Loss: 1.587 | Train Acc: 56.55%
	 Val. Loss: 0.951 |  Val. Acc: 73.17%
Epoch: 03 | Epoch Time: 0m 25s
	Train Loss: 0.840 | Train Acc: 76.42%
	 Val. Loss: 0.668 |  Val. Acc: 81.69%
Epoch: 04 | Epoch Time: 0m 24s
	Train Loss: 0.610 | Train Acc: 82.76%
	 Val. Loss: 0.605 |  Val. Acc: 83.77%
Epoch: 05 | Epoch Time: 0m 24s
	Train Loss: 0.509 | Train Acc: 85.46%
	 Val. Loss: 0.574 |  Val. Acc: 84.41%
Epoch: 06 | Epoch Time: 0m 25s
	Train Loss: 0.430 | Train Acc: 87.77%
	 Val. Loss: 0.566 |  Val. Acc: 84.91%
Epoch: 07 | Epoch Time: 0m 25s
	Train Loss: 0.370 | Train Acc: 89.47%
	 Val. Loss: 0.587 |  Val. Acc: 84.29%
Epoch: 08 | Epoch Time: 0m 25s
	Train Loss: 0.323 | Train Acc: 90.77%
	 Val. Loss: 0.570 |  Val. Acc: 85.63%
Epoch: 09 | Epoch Time: 0m 26s
	Train Loss: 0.284 | Train Acc: 91.76%
	 Val. Loss: 0.593 |  Val. Acc: 85.26%
Epoch: 10 | Epoch T

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Project 3/Social media text/tweet_BERT.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.565 | Test Acc: 84.13%


In [ ]:
def tag_sentence(model, device, sentence, tokenizer, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        tokens = tokenizer.tokenize(sentence)
    else:
        tokens = sentence
    
    numericalized_tokens = tokenizer.convert_tokens_to_ids(tokens)
    numericalized_tokens = [text_field.init_token] + numericalized_tokens
        
    unk_idx = text_field.unk_token
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    predicted_tags = predicted_tags[1:]
        
    assert len(tokens) == len(predicted_tags)
    
    return tokens, predicted_tags, unks

In [ ]:
sentence2 = 'My heart is always breaking for the ghosts that haunt this room.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence2,
                                  tokenizer,
                                  TEXT, 
                                  TWEET_TAGS)

print(unks)

[]


In [ ]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

D		my
N		heart
V		is
R		always
V		breaking
P		for
D		the
N		ghosts
P		that
V		haunt
D		this
N		room
,		.
